In [ ]:
DATA_PATH = "../Phase 3/Jobenn_preprocessed_NCF_data.csv"
df = pd.read_csv(DATA_PATH, encoding='utf-8')

# df.head()
import random
random.seed(42)
df = df.sample(10)
print(df.dtypes) # I want to see the type of the features
print(df['label'])
print(df['text'])

Hopefully its the last


In [ ]:
## Sibusiso Maduna 
## u21565687

# ==============================
# XAI WITH SHAP + LIME 
# ==============================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
from lime.lime_text import LimeTextExplainer
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)

# Config
BERT_DIR = "./Tuned Models/bert_finetuned"
ROBERTA_DIR = "./Tuned Models/roberta_finetuned"
DATA_PATH = "../Phase 3/Jobenn_preprocessed_NCF_data.csv"

OUTPUT_DIR = "Final_xai_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SHAP_EXPLANATIONS = 2
LIME_EXPLANATIONS = 2
TEST_SAMPLES = 50

# ==============================
# FIXED DATA LOADING
# ==============================

def debug_load_data():
    print("\n🔍 DEBUG: Loading structured CSV and constructing input texts...")
    try:
        df = pd.read_csv(DATA_PATH, encoding='utf-8')
        print(f"✅ Loaded shape: {df.shape}")
        print(f"🔎 Columns: {df.columns.tolist()}")

        if df.isna().all().all():
            raise ValueError("❌ All data is NaN - Check the CSV format.")

        candidate_columns = ['protocol', 'flag', 'family', 'address', 'btc', 'netflow_bytes', 'segment']
        texts = []
        for _, row in df.iterrows():
            parts = []
            for col in candidate_columns:
                if col in df.columns and pd.notna(row[col]):
                    val = row[col]
                    parts.append(f"{col}: {val}" if isinstance(val, (int, float)) else str(val))
            texts.append(" ".join(parts) if parts else "missing_text")

        if 'label' in df.columns:
            label_col = 'label'
        elif 'category' in df.columns:
            label_col = 'category'
        else:
            raise ValueError("❌ No label or category column found.")

        labels = df[label_col].apply(lambda l: 1 if str(l).strip().lower() in ['1', 'ransomware', 'malicious'] else 0).tolist()

        if len(texts) > TEST_SAMPLES:
            texts = texts[:TEST_SAMPLES]
            labels = labels[:TEST_SAMPLES]

        return texts, labels

    except Exception as e:
        print(f"❌ Exception in data loading: {e}")
        exit()

# ==============================
# TEXT-BASED SHAP MULTIPLE SAMPLE PLOTS
# ==============================

def shap_text_multi_sample(model, tokenizer, device, texts, model_name):
    print(f"\n🔄 SHAP Text Explanations for {model_name}...")

    # --- Robust coercion: anything -> list[str] ---
    def _to_text_list(x):
        if isinstance(x, np.ndarray):
            x = x.ravel().tolist()
        elif isinstance(x, pd.Series):
            x = x.tolist()
        elif not isinstance(x, (list, tuple)):
            x = [x]
        out = []
        for t in x:
            if isinstance(t, str):
                out.append(t)
            elif isinstance(t, (list, tuple, np.ndarray)):
                out.append(" ".join(map(str, t)))
            else:
                out.append(str(t))
        return out

    # --- Batch predictor returning (batch, n_classes) ---
    def predict_proba(texts_like):
        texts_list = _to_text_list(texts_like)
        enc = tokenizer(texts_list, return_tensors="pt", truncation=True, padding=True, max_length=128)
        if device != -1:
            enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            logits = model(**enc).logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()
        return probs

    masker = shap.maskers.Text(tokenizer)
    explainer = shap.Explainer(predict_proba, masker, output_names=["Benign", "Ransomware"])

    k = min(SHAP_EXPLANATIONS, len(texts))
    sample_texts = texts[:k]

    # Compute explanations for multiple samples (Explanation of length k)
    exps = explainer(sample_texts, silent=True)

    # 1) Per-sample HTML + per-sample bar/waterfall (class 1)
    for i in range(k):
        e = exps[i]                     # single-sample Explanation
        # slice to class 1 safely
        e1 = e[..., 1] if getattr(e, "values", None) is not None and np.ndim(e.values) == 2 else e
        # HTML text plot
        try:
            vis = shap.plots.text(e1, display=False)
            out_html = f"{OUTPUT_DIR}/shap_text_sample_{i}_{model_name}.html"
            with open(out_html, "w", encoding="utf-8") as f:
                f.write(getattr(vis, "data", vis))
            print(f"✅ SHAP text saved: {out_html}")
        except Exception as err:
            print(f"⚠️ Could not save SHAP text HTML for sample {i}: {err}")

        # Per-sample bar
        try:
            plt.figure(figsize=(10, 6))
            shap.plots.bar(e1, show=False, max_display=20)
            plt.title(f"{model_name} – Sample {i} – Ransomware", fontsize=13)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/shap_bar_{model_name}_sample{i}.png", dpi=150, bbox_inches="tight")
            plt.close()
        except Exception as err:
            print(f"⚠️ Per-sample bar failed (sample {i}): {err}")

        # Optional waterfall
        try:
            plt.figure(figsize=(10, 6))
            shap.plots.waterfall(e1, show=False, max_display=20)
            plt.title(f"{model_name} – Waterfall – Sample {i}", fontsize=13)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/shap_waterfall_{model_name}_sample{i}.png", dpi=150, bbox_inches="tight")
            plt.close()
        except Exception as err:
            print(f"⚠️ Waterfall failed (sample {i}): {err}")

    # 2) Aggregate top tokens across samples (manual Matplotlib bar chart)
    try:
        from collections import defaultdict
        token_scores = defaultdict(float)

        for i in range(k):
            e = exps[i]
            vals = e.values
            # get per-token contribs for class 1
            per_token = vals[:, 1] if np.ndim(vals) == 2 else np.asarray(vals).ravel()
            # tokens (text features)
            toks = e.data
            # e.data may be nested; coerce to a flat list of strings
            if isinstance(toks, (list, tuple, np.ndarray)):
                tokens_list = [str(t) for t in (toks.tolist() if isinstance(toks, np.ndarray) else toks)]
            else:
                tokens_list = [str(toks)]
            # accumulate mean |shap|
            for t, s in zip(tokens_list, per_token):
                token = t.strip()
                if token:  # skip empty
                    token_scores[token] += float(abs(s))

        # top-N tokens
        topN = 25
        top_items = sorted(token_scores.items(), key=lambda kv: kv[1], reverse=True)[:topN]
        if top_items:
            tokens, scores = zip(*top_items)
            plt.figure(figsize=(12, 8))
            y = np.arange(len(tokens))[::-1]
            plt.barh(y, scores[::-1])
            plt.yticks(y, [t for t in tokens[::-1]])
            plt.xlabel("Aggregate |SHAP|")
            plt.title(f"{model_name} – Aggregate Top Tokens (Ransomware)")
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/shap_aggregate_tokens_{model_name}.png", dpi=150, bbox_inches="tight")
            plt.close()
            print(f"✅ Aggregate token bar saved: {OUTPUT_DIR}/shap_aggregate_tokens_{model_name}.png")
        else:
            print("⚠️ No tokens collected for aggregate plot.")
    except Exception as err:
        print(f"⚠️ Aggregate token plot failed: {err}")
# ==============================
# LIME RE-INTEGRATION (FIXED)
# ==============================

def simple_lime_analysis(model, tokenizer, device, texts, model_name):
    print(f"\n🔄 LIME for {model_name}...")

    def predict_proba(texts_list):
        all_probs = []
        for text in texts_list:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding=True)
            if device != -1:
                inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
                probs = torch.softmax(outputs.logits, dim=-1)[0].cpu().numpy()
            all_probs.append(probs)
        return np.array(all_probs)

    explainer = LimeTextExplainer(class_names=["Benign", "Ransomware"])
    success = 0

    for i, text in enumerate(texts[:LIME_EXPLANATIONS]):
        try:
            exp = explainer.explain_instance(
                text, predict_proba, num_features=5, num_samples=50
            )
            exp.save_to_file(f"{OUTPUT_DIR}/lime_{model_name}_sample{i}.html")
            
            # Create matplotlib figure
            plt.figure(figsize=(10, 6))
            exp.as_pyplot_figure()
            plt.title(f"LIME - {model_name} - Sample {i}", fontsize=14)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/lime_{model_name}_sample{i}.png", dpi=120)
            plt.close()
            
            # Show the predictions with table (FIXED VERSION)
            print(f"\n📋 LIME Predictions for {model_name} - Sample {i}:")
            print(f"Text: {text[:100]}...")
            
            # Get prediction probabilities
            probs = predict_proba([text])[0]
            print(f"Prediction probabilities: Benign: {probs[0]:.4f}, Ransomware: {probs[1]:.4f}")
            
            # Show LIME explanation as text
            print("\nLIME Feature Importance:")
            for feature, weight in exp.as_list():
                print(f"  {feature}: {weight:.4f}")
            
            # Create a simple text-based table
            print("\nLIME Explanation Table:")
            print("Feature".ljust(30) + "Weight".ljust(15))
            print("-" * 45)
            for feature, weight in exp.as_list():
                print(f"{feature[:29].ljust(30)}{weight:+.4f}")
            
            success += 1
            
        except Exception as e:
            print(f"❌ LIME failed for sample {i}: {e}")

    return success

# ==============================
# MODEL LOADING
# ==============================

def load_model(model_dir):
    print(f"🔧 Loading model from {model_dir}")
    if not os.path.exists(model_dir):
        print(f"❌ Directory not found: {model_dir}")
        return None, None, None

    try:
        device = 0 if torch.cuda.is_available() else -1
        tokenizer = AutoTokenizer.from_pretrained(model_dir)
        model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        model.eval()
        if device != -1:
            model = model.to(device)
        return model, tokenizer, device
    except Exception as e:
        print(f"❌ Loading failed: {e}")
        return None, None, None

# ==============================
# MAIN EXECUTION
# ==============================

def main():
    print("\n=== FIXED XAI EXECUTION ===")
    texts, labels = debug_load_data()

    models = {
        "BERT": BERT_DIR,
        "RoBERTa": ROBERTA_DIR
    }

    for model_name, model_dir in models.items():
        print(f"\n{'='*50}\nPROCESSING: {model_name}\n{'='*50}")

        model, tokenizer, device = load_model(model_dir)
        if model is None:
            continue

        try:
            inputs = tokenizer(texts[0], return_tensors="pt", truncation=True)
            if device != -1:
                inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
                print(f"✅ Test prediction: {torch.softmax(outputs.logits, dim=-1)}")
        except Exception as e:
            print(f"❌ Test prediction failed: {e}")
            continue

        shap_text_multi_sample(model, tokenizer, device, texts, model_name)
        simple_lime_analysis(model, tokenizer, device, texts, model_name)

if __name__ == "__main__":
    main()


=== FIXED XAI EXECUTION ===

🔍 DEBUG: Loading structured CSV and constructing input texts...
✅ Loaded shape: (20186, 26)
🔎 Columns: ['protocol', 'flag', 'family', 'address', 'usd', 'seed_address', 'btc', 'netflow_bytes', 'ip_address', 'clusters', 'threats', 'port', 'time', 'prediction', 'label', 'text', 'segment', 'embeddings', 'dataset', 'r', 'rw', 'rx', 'rwc', 'rwx', 'rwxc', 'category']

PROCESSING: BERT
🔧 Loading model from ./Tuned Models/bert_finetuned
✅ Test prediction: tensor([[9.9974e-01, 2.6269e-04]])

🔄 SHAP Text Explanations for BERT...
✅ SHAP text saved: Final_xai_outputs/shap_text_sample_0_BERT.html
⚠️ Per-sample bar failed (sample 0): string index out of range
✅ SHAP text saved: Final_xai_outputs/shap_text_sample_1_BERT.html
⚠️ Per-sample bar failed (sample 1): string index out of range
✅ Aggregate token bar saved: Final_xai_outputs/shap_aggregate_tokens_BERT.png

🔄 LIME for BERT...

📋 LIME Predictions for BERT - Sample 0:
Text: TCP APS Razy 1DiCeTjB btc: 0.37478141634155

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [ ]:
import pandas as pd
df = pd.read_csv("../Phase 3/Jobenn_preprocessed_NCF_data.csv")
print(df.columns.tolist())
df['text']